In [1]:
import numpy as np
import pandas as pd

# Importamos pyplot
from matplotlib import pyplot as plt

In [2]:
catalog = "../../Bases_de_datos/Data_catalog.csv"
iso = "../../Bases_de_datos/Country_ISO_code.csv"
path_v = "../data/variables.csv"
ix = "alpha-3"

df_c = pd.read_csv(catalog)
df_iso = pd.read_csv(iso).set_index(ix)
data_0 = pd.DataFrame(index = df_iso.index)
data_s   = data_0.copy()
data_c   = data_0.copy()
data_g   = data_0.copy()
data_q_s = data_0.copy()
data_q_c = data_0.copy()
data_q_g = data_0.copy()

df_v = pd.read_csv(path_v)

In [3]:
for cat in df_v["Category"].unique():
    data   = data_0.copy()
    data_q = data_0.copy()

    for row in df_v[ (df_v["Category"] == cat) ].itertuples():

        v = row.Variable
        if row.Origin == "World Bank": nr = 3
        else: nr = 0
        
        df = pd.read_csv( "../../"
            + df_c.loc[df_c["ID"]==row.ID, "Path"].iloc[0]
            + df_c.loc[df_c["ID"]==row.ID, "Filename" ].iloc[0],
            skiprows = nr, low_memory = False )
        
        if row.Origin == "OECD":
            df = df[ df["MEASURE"] == v ]
            df = df.rename(columns = {"REF_AREA": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            
            df_i = df[ (df["MEASURE"]==v) ]
            
            if v == "POP_NETMIGR":
                df_i["OBS_VALUE"] = np.abs( df_i["OBS_VALUE"] )
            
            elif v in ["HD_POP_EXP", "HD_TN_POP_EXP", "ID_POP_EXP",
                "99PT_LAND_EXP", "99PT_CROP_EXP", "EP_LAND_EXP"]:
                df_i = df_i[ (df_i["DURATION"]==row.DURATION) ]
            elif v in ["TEMP_CHANGE", "ED_CROP_ANOM", "EP_CHANGE"]:
                df_i = df_i.set_index(ix)
                df_i["OBS_VALUE"] = df_i.reset_index()[
                    [ix, "OBS_VALUE"] ].groupby(ix).mean()
                df_i = df_i.reset_index()
            elif v == "UTCI_POP_EXP":
                df_i = df_i[ (df_i["HEAT_STRESS"]==row.HEAT_STRESS) ]
            elif v in ["CF_BUILT_EXP", "CF_POP_EXP", "CF_CROP_EXP",
                "RF_BUILT_EXP", "RF_POP_EXP", "RF_CROP_EXP"]:
                df_i = df_i[ (df_i["RET_PERIOD"]==row.RET_PERIOD) ]
            elif v == "BURN_LAND_AREA":
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            elif v in ["CY_BUILT_EXP", "CY_POP_EXP"]:
                df_i = df_i[
                    (df["HURRICANE_WIND_SCALE"]==row.HURRICANE_WIND_SCALE) ]
            elif v == "POP_EXP_POL":
                df_i = df_i[ (df_i["EXPOSURE_LEVEL"]==row.EXPOSURE_LEVEL) ]

            elif v == "ECRATE":
                df_i = df_i[ (df_i["SECTOR"]==row.SECTOR)
                & (df["EMISSIONS_SOURCE"]==row.EMISSIONS_SOURCE) ]
            elif v in ["GPAT_DE", "WATER_FW"]:
                df_i = df_i[ (df_i["UNIT_MEASURE"]==row.UNIT_MEASURE) ]
            
            data[v] = df_i.sort_values( [ix, "TIME_PERIOD"]
                ).drop_duplicates(ix, keep="last").set_index(ix)["OBS_VALUE"]

        elif row.Origin == "Global Data Lab":
            df = df.rename(columns = {"ISO_Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df[ df["Level"] == "National" ].set_index(ix)[v]
        
        elif row.Origin == "World Bank":
            df = df.rename(columns = {"Country Code": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            data[v] = df.set_index(ix).iloc[ :, 4:-1
                ].ffill(axis = 1).iloc[:, -1]
        
        elif row.Origin == "UNESCO UIS":
            if row.ID in ["DEM", "SCN-SDG", "SDG11"]:
                df.columns = [ x.lower() for x in df.columns ]
            df = df.rename(columns = {"country_id": ix})
            df = df[ df[ix].isin(df_iso.index) ]
            if row.PERCENTAGE == "Yes": pop = data["200101"]
            else: pop = 1
            data[v] = ( df[ df["indicator_id"]==v ].sort_values( [ix, "year"]
                ).drop_duplicates(ix, keep="last"
                ).set_index(ix)["value"] / pop )
        
        # corregir random
        data_q[v] = ( 1 + pd.qcut( data[v]
            + np.random.random(data[v].shape[0])/10000, 20, labels = False ) )
        if row.Negative_indicator:
            data_q[v] = 21 - data_q[v]

    data_q = data_q / 2
    data_q["Mean"] = data_q.mean(axis = 1)
    data_q["Count"] = data_q.count(axis = 1)
    data_q = data_q[ data_q["Count"] > 1 ].sort_values("Mean")

    if   cat == df_v["Category"].unique()[0]:
        data_q_s = data_q.copy()
        data_s   = data.copy()
        data_q_s.to_csv("../results/Deciles_Socioeconomico.csv", index = True)
        data_c.to_csv("../results/Valores_Socioeconomico.csv", index = True)
    elif cat == df_v["Category"].unique()[1]:
        data_q_c = data_q.copy()
        data_c   = data.copy()
        data_q_c.to_csv("../results/Deciles_Climatico.csv", index = True)
        data_c.to_csv("../results/Valores_Climatico.csv", index = True)
    elif cat == df_v["Category"].unique()[2]:
        data_q_g = data_q.copy()
        data_g   = data.copy()
        data_q_g.to_csv("../results/Deciles_Gobernanza.csv", index = True)
        data_g.to_csv("../results/Valores_Gobernanza.csv", index = True)

data_q_c

,HD_POP_EXP,HD_TN_POP_EXP,ID_POP_EXP,UTCI_POP_EXP,TEMP_CHANGE,CF_BUILT_EXP,CF_POP_EXP,CF_CROP_EXP,ED_CROP_ANOM,99PT_LAND_EXP,...,FT_FORE_EXP,CY_BUILT_EXP,CY_POP_EXP,W_BUILT_EXP,W_POP_EXP,MEAN_POP,POP_EXP_POL,EP_LAND_EXP,Mean,Count
alpha-3,,,,,,,,,,,,,,,,,,,,,
MLI,0.5,0.5,2.0,1.0,4.0,NaN,NaN,NaN,8.5,3.0,...,0.5,0.5,3.5,6.0,5.5,0.5,0.5,5.5,2.727273,23
IND,1.0,1.0,2.0,1.0,7.0,3.0,6.0,3.5,0.5,1.0,...,3.0,3.5,3.0,7.5,5.5,1.0,1.5,3.0,2.780000,26
THA,2.5,2.5,2.0,3.0,6.0,1.0,2.5,2.5,2.5,5.0,...,5.5,2.0,1.5,8.0,8.5,2.0,2.0,2.0,3.020000,26
EGY,1.0,1.0,3.0,1.5,2.5,1.0,2.0,0.5,1.0,9.0,...,1.0,6.5,2.5,8.5,6.5,0.5,1.0,7.5,3.060000,26
IRQ,0.5,0.5,9.0,0.5,1.0,3.0,4.5,4.0,1.5,3.0,...,1.0,3.5,1.0,3.5,6.0,1.5,1.5,6.5,3.200000,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAF,9.5,4.5,7.0,NaN,9.5,NaN,NaN,NaN,NaN,10.0,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,7.900000,11
BES,8.0,7.0,5.5,NaN,9.0,NaN,NaN,NaN,NaN,6.0,...,9.5,NaN,NaN,NaN,NaN,NaN,NaN,7.5,7.950000,11
MSR,10.0,10.0,2.5,8.0,10.0,NaN,10.0,9.5,NaN,7.5,...,8.5,NaN,6.5,NaN,7.5,9.0,8.5,8.5,8.100000,21


In [4]:
res_names = [ "Socioeconomic resilience",
    "Climate resilience", "Governance resilience" ]

df_iso[ res_names[0]  ] = data_q_s.loc[
    data_q_s["Count"] > 11, "Mean" ]
df_iso[ res_names[1]  ] = data_q_c.loc[
    data_q_c["Count"] > 11, "Mean" ]
df_iso[ res_names[2]  ] = data_q_g.loc[
    data_q_g["Count"] > 3, "Mean" ]

df_nz = pd.read_csv( "../../" +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Path" ].iloc[1] +
    df_c.loc[ df_c["Origin"]=="Climate Action Tracker", "Filename" ].iloc[1],
    usecols = ["ISO", "Overall Rating"] )
df_nz = df_nz.rename( columns = {"ISO": ix} ).set_index(ix)
df_iso["Net Zero Rating"] = df_nz["Overall Rating"]
df_iso["Net Zero Rating"] = df_iso["Net Zero Rating"].where(
    ~df_iso["Net Zero Rating"].isnull(), "No target" )

df_res = df_iso.drop( ["country-code", "iso_3166-2", "intermediate-region",
    "region-code", "sub-region-code", "intermediate-region-code"], axis = 1 )

df_res

df_rnz = df_res.loc[ df_res["Net Zero Rating"] != "No target",
    ["name", "Net Zero Rating", "alpha-2"] + res_names ]

df_rnz.to_csv( "../results/resumen_categorias.csv", index = True )

df_rnz

,name,Net Zero Rating,alpha-2,Socioeconomic resilience,Climate resilience,Governance resilience
alpha-3,,,,,,
ARG,Argentina,Poor,AR,5.617647,3.720000,5.555556
AUS,Australia,Poor,AU,6.394737,4.960000,5.240000
BTN,Bhutan,Information incomplete,BT,5.978261,5.136364,3.687500
BRA,Brazil,Information incomplete,BR,5.812500,4.600000,5.880952
CAN,Canada,Average,CA,6.138889,4.360000,5.108696
CHL,Chile,Acceptable,CL,5.913043,4.980000,6.204545
CHN,China,Poor,CN,4.956522,3.720000,4.976190
COL,Colombia,Acceptable,CO,4.680000,4.780000,6.086957
CRI,Costa Rica,Acceptable,CR,6.520000,5.180000,5.434783


In [5]:
import plotly.express as px
import plotly.graph_objects as go

df_g = df_rnz.copy()

n = list(range(df_rnz.shape[0]))
n.remove(0)
n.remove(2)
n.remove(4)
n.remove(5)
df_g = df_g.iloc[[2, 0, 4, 5] + n ]
for i in res_names:
    df_g[i] = df_rnz[i].apply(lambda x: float(f"{x:.2f}"))


fig = px.scatter( df_g, x = res_names[0], y = res_names[1],
    hover_name = "name", hover_data = res_names + ["Net Zero Rating"],
    size = res_names[2], color = "Net Zero Rating", opacity = 1, size_max = 25,
    color_discrete_sequence=["#808080", "#8B0000", "#DAA520", "#006400"]
)
#fig.update_traces(marker_color="rgba(0,0,0,0)")

for i, row in df_rnz.iterrows():
    country_iso = row["alpha-2"]
    fig.add_layout_image(
        dict(
            source=f"https://raw.githubusercontent.com/matahombres/CSS-Country-Flags-Rounded/master/flags/{country_iso}.png",
            xref="x", yref="y", xanchor="center", yanchor="middle",
            x=row[res_names[0]], y=row[res_names[1]],
            sizex=row[res_names[2]]/25, sizey=row[res_names[2]]/25,
            sizing="contain",
            #bordercolor = "green",
            opacity=1,
            layer="above"
        )
    )

fig.update_layout(height=600, width=1000, plot_bgcolor="#dfdfdf")

fig.show()